In [48]:
# !pip install selenium-stealth
# !pip install fake-useragent

### Run it to intialize your browser

In [1]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import json
import pandas as pd


def start():
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_argument("--headless")  # Uncomment if you want to run headless

    # Fake User-Agent
    ua = UserAgent()
    user_agent = ua.random
    options.add_argument(f"user-agent={user_agent}")

    # Disable webdriver mode
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(options=options)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    return driver



### Extract company links

In [ ]:

driver = start()
company_links = set()


url = "https://clutch.co/developers?page=1"
driver.get(url)
request = driver.page_source
soup = BeautifulSoup(request, 'html.parser')

for i in range(708):  # here 708 is no. of page
    script_tag = soup.find_all("script",type="application/ld+json")

    json_data_company = [
        json.loads(x.string) for x in script_tag
    ]

    for d in json_data_company:
        if d['@type']=='Organization':
            company_links.add(d['url']+'#highlights')
        elif d['@type'] == 'CollectionPage':
            list_elements = d['mainEntity']['itemListElement']
            for item in list_elements:
                if item['item']['@type'] == 'Organization':
                    company_links.add(item['item']['url']+'#highlights')
    driver.quit()
    time.sleep(5)
    driver = start()
    if i+1 != 1:
        url = url.split('=')[0] + f'={i+1}'
        driver.get(url)
        request = driver.page_source
        soup = BeautifulSoup(request, 'html.parser')

### Saving Links to excel file

In [24]:
# Create a DataFrame from the array
df = pd.DataFrame(company_links, columns=["Links"])

# Save the DataFrame to an Excel file
df.to_excel("links.xlsx", index=False)

print("Links saved to links.xlsx")

Links saved to links.xlsx


### Reading Links from excel file

In [2]:
# Read the Excel file into a DataFrame
df = pd.read_excel("links.xlsx")

# Convert the DataFrame column back to a list
company_links = df["Links"].tolist()

print("Links read from links.xlsx:")

Links read from links.xlsx:


### Now start extracting reviewer's data

In [15]:
data_list = []
driver = start()
company_no = 8000
starting = 8000
for k in range(starting,8100): ## till 35 thousands
    url = company_links[k]
    driver = start()
    driver.get(url)
    time.sleep(3)
    bs = BeautifulSoup(driver.page_source,"html.parser")
    try:
        try:
            max_page = bs.find('span',{'class':'sg-pagination__text'})
            max_page = max_page.text.split('of')[1].strip()
            max_page = int(max_page)
        except:
            max_page = 1
        

        for j in range(max_page):
            for div in bs.find_all('div',{'class': "profile-review__reviewer"}):
                data = {}
                name = div.find('div',{'class':'reviewer_card--name'}).text
                postion = div.find('div',{'class':'reviewer_position'}).text
                domain = div.find_all('span',{'class':'reviewer_list__details-title'})
                data['name']=name 
                data['position']=postion
                data['domain'] =domain[0].text
                if 'Employee' in domain[1].text:
                    data['Employee NO']= domain[1].text
                    data['location'] = domain[2].text
                elif 'Employee' in domain[2].text:
                    data['Employee NO']= domain[2].text
                    data['location'] = domain[1].text

                data_list.append(data)
            driver.quit()
            
            if j+1 !=1:
                driver = start()
                url = f'?page={j+1}#'.join("https://clutch.co/profile/altario#highlights".split('#'))
                driver.get(url)
                bs = BeautifulSoup(driver.page_source,"html.parser")
                
        
    except:
        driver.quit()
        time.sleep(2)
        print("Problem with ---->",url)
    
    print(company_no)
    company_no+=1
    
            

8000
8001
8002
8003
8004
8005
8006
8007
8008
8009
8010
8011
8012
8013
8014
8015
8016
8017
8018
8019
8020
8021
8022
8023
8024
8025
8026
8027
8028
8029
8030
8031
8032
8033
8034
8035
8036
8037
8038
8039
8040
8041
8042
8043
8044
8045
8046
8047
8048
8049
8050
8051
8052
8053
8054
8055
8056
8057
8058
8059
8060
8061
8062
8063
8064
8065
8066
8067
8068
8069
8070
8071
8072
8073
8074
8075
8076
8077
8078
8079
8080
8081
8082
8083
8084
8085
8086
8087
8088
8089
8090
8091
8092
8093
8094
8095
8096
8097
8098
8099


### Convert reviewer's data to table for visualization

In [16]:
data_list = [dict(t) for t in {tuple(d.items()) for d in data_list}]
df = pd.DataFrame(data_list)
df

,name,position,domain,Employee NO,location
0,Larysa Kruk,"Project Manager, Audio Tour Mobile App",Software,1-10 Employees,"Amsterdam, Netherlands"
1,Karmand Ahmad,"Ganeral Manager, ZMC Cargo",Other Industry,201-500 Employees,"Erbil, Iraq"
2,Prashant Parmar,"President, Aspect Commercial Realty & Business...",Real estate,1-10 Employees,"Charlotte, North Carolina"
3,Anonymous,"Operational Director, InDhan PayGate",Software,11-50 Employees,"Indore, India"
4,Anonymous,"CEO, Venture Company",Consumer Products,11-50 Employees,"London, United Kingdom"
...,...,...,...,...,...
88,Andrea Mattei,"Marketing Manager, Atena S.p.A.",Consulting,51-200 Employees,"Brescia, Italy"
89,Adrian Ireland,"Founder & CEO, Educational Institution",Education,1-10 Employees,"Dover, Delaware"
90,Arel Sirin,"CTO & Founder, Sunshift Dev",IT Services,11-50 Employees,"Montevideo, Uruguay"
91,Anonymous,"Vice President, Macro Polymers Pvt Ltd",Chemicals & Biotechnology,201-500 Employees,"Ahmedabad, India"


### Append data in previous excel file

In [17]:
df1 = pd.read_excel('output.xlsx')
df1 = pd.concat([df1, df], ignore_index=True, sort=False)
df1=df1[df1['name']!='Anonymous'].drop_duplicates()

In [18]:
# Export to Excel
excel_file_path = 'output.xlsx'
df1.to_excel(excel_file_path, index=False)

print(f"Data exported to {excel_file_path}")
print(f'Total data till now {len(df1)}')

Data exported to output.xlsx
Total data till now 6432
